In [14]:
import pandas as pd
import numpy as np
import re

from sklearn.metrics.pairwise import cosine_similarity

In [15]:
def string_to_vector(string):
  """
  Turns a string of a list of numbers into a list of numbers
  """
  string = re.sub("[\[\]]", "", string)
  string = re.sub("\s+", " ", string)
  string = string.split(' ')
  return np.array([float(x) for x in string if x != ""])

In [16]:
full_df = pd.read_csv(r"C:\Users\asarr\Documents\MACSS\Thesis\results\full_df.csv", index_col=0)
full_df.head()

,session_id,date,chamber,intervention_id,intervention_text,intervention_words,embeddings,cluster,num_words,year,svd_embeddings,graph_embeddings
0,gaceta_459 (7),2006-10-18,senate,452004,(julio 21) por medio de la cual se crea una c...,"['accidental', 'acreditación', 'acrediten', 'a...",[ 2.75705218e-01 -1.56717077e-01 5.24776168e-...,5,21,2006,[-1.07373972e+01 -5.24858566e-02 -3.47477753e-...,[-10.73739721 -0.05248475]
1,gaceta_459 (7),2006-10-18,senate,452003,(septiembre 11) por medio de la cual se aclar...,"['aclara', 'aclárese', 'acto', 'administrativo...",[ 4.02346551e-01 -1.51027605e-01 2.49001503e-...,1,26,2006,[-1.11914571e+01 1.63042020e+00 1.67238361e+...,[-11.19145712 1.63042081]
2,gaceta_459 (7),2006-10-18,senate,452002,(septiembre 8) por medio de la cual se design...,"['artículo', 'comisión', 'congreso', 'convenio...",[ 1.86938807e-01 -1.24048591e-01 4.23117690e-...,5,16,2006,[-1.12215688e+01 -2.87521366e-01 -6.22032624e-...,[-11.22156882 -0.2875204 ]
3,gaceta_459 (7),2006-10-18,senate,452001,si bien el crecimiento económico es un factor ...,"['aumentado', 'crecimiento', 'distribución', '...",[ 5.93414120e-02 -1.13201797e-01 1.75440997e-...,5,21,2006,[-1.10547869e+01 -2.22703892e+00 -2.72106466e+...,[-11.05478689 -2.22703888]
4,gaceta_459 (7),2006-10-18,senate,451999,fuente: merpd la disminución en la inequidad ...,"['aquellos', 'aumentado', 'caído', 'coeficient...",[ 4.15660948e-01 -1.86677556e-02 3.19064856e-...,2,33,2006,[-1.08555325e+01 -1.99172156e+00 -1.98372695e+...,[-10.85553251 -1.99172263]


In [17]:
full_df['embeddings'] = full_df['embeddings'].apply(string_to_vector)

In [18]:
labels = list(set(full_df['cluster'].to_list()))
labels

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [21]:
import warnings
warnings.filterwarnings('ignore')

representative_interventions = {f'cluster_{label}': [] for label in labels if label != 0 and label != 4} 
for cluster in labels:
    if cluster == 0 or cluster == 4:
        continue
    cluster_df = full_df[full_df['cluster'] == cluster]
    avg_vector = np.mean(cluster_df['embeddings'].values, axis=0)
    cluster_df['similarity'] = cluster_df['embeddings'].apply(lambda x: cosine_similarity([avg_vector], [x])[0][0])
    cluster_df = cluster_df.sort_values(by='similarity', ascending=False)
    representative_interventions[f'cluster_{cluster}'] = cluster_df['intervention_text'].to_list()[:20]

In [24]:
representative_interventions_df = pd.DataFrame(representative_interventions)
representative_interventions_df.to_csv(r"C:\Users\asarr\Documents\MACSS\Thesis\results\representative_interventions.csv")